# Validatore GTF
**Avenoso Luigi 844787**

##### Testo esercizio
Si richiede di scrivere un validatore del formato GTF (Gene Transfer Format) che prenda in input un file in formato GTF che annota un set di geni ed effettui la validazione del file rispetto alla specifica del formato (<https://mblab.wustl.edu/GTF22.html>).
Il validatore deve produrre in output un report con le violazioni presenti, specificando per ognuna di esse il record che la contiene (posizione all'interno del file in input) e tutte le informazioni che si ritengono necessarie per descriverla e correggerla.
Il validatore può essere prodotto sia come script che come Jupyter Notebook, e deve essere adeguatamente commentato. Si richiede inoltre un documento che elenchi e descriva brevemente le violazioni che sono state considerate. Per ogni violazione considerata, includere un file con tale violazione.
Il validatore deve essere caricato in un repository GitHub di cui va consegnato il link

**La validazione è stata sviluppata solamente a livello sintattico**

In [1]:
import re
import collections

### File disponibili per i test
Per eseguire i test è necessario decommentare il file gtf relativo, commentando l’attuale

In [2]:
# Contiene l'errore sul numero dei campi 
#gtf_file_name = "./file_test/test1.gtf"

# Contiene l’errore nel campo seqname
#gtf_file_name = "./file_test/test2.gtf"

# Contiene l’errore nel campo source
#gtf_file_name = "./file_test/test3.gtf"

# Contiene l’errore nel campo feature
#gtf_file_name = "./file_test/test4.gtf"

# contiene l’errore nei campi start ed end
#gtf_file_name = "./file_test/test5.gtf"

# Contiene l’errore nel campo score
#gtf_file_name = "./file_test/test6.gtf"

# Contiene l’errore nel campo strand
#gtf_file_name = "./file_test/test7.gtf"

# Contiene l’errore nel campo frame
#gtf_file_name = "./file_test/test8.gtf"

# Contiene l’errore nel campo attributes
#gtf_file_name = "./file_test/test9.gtf"

# File corretto con presenza di commenti
#gtf_file_name = "./file_test/test10.gtf"

# Contiene tutti i possibili errori dei file precedenti
gtf_file_name = "./file_test/test11.gtf"

**Lettura file GTF**

In [3]:
# Lettura file GTF 

with open(gtf_file_name, 'r') as gtf_input_file:
    gtf_file_rows = gtf_input_file.readlines()

In [4]:
gtf_file_rows

In [5]:
# Eliminazione degli eventuali commenti(#) dal file
# I commenti possono verificarsi ovunque nel file, anche alla fine di una linea caratteristica
# Sono stai considerati anche commenti che, alla fine della linea, sono separati da un TAB dall'ultimo campo
# Esempio: transcript_id "U52112.4-005"; gene_id "ARHGAP4";\t#commento

# Sono stai considerati anche commenti che, alla fine della linea, non sono separati da un TAB dall'ultimo campo
# Esempio: transcript_id "U52112.4-005"; gene_id "ARHGAP4";#commento

for row in gtf_file_rows:
    if '\t#' in row:
        gtf_file_rows[gtf_file_rows.index(row)] = gtf_file_rows[gtf_file_rows.index(row)][:row.index('\t#')]
    elif '#' in row:
        gtf_file_rows[gtf_file_rows.index(row)] = gtf_file_rows[gtf_file_rows.index(row)][:row.index('#')]

In [6]:
gtf_file_rows

#### Funzioni per il controllo sintattico del file
*seq_id(gtf_rows)*
- Ritorna il dict (gene_id, seqname)
- La correttezza del campo **seqname** si basa su una statistica
- Assumo che il campo **seqname** più numeroso associato al relativo gene sia quello corretto per il gene
- Il numero maggiore di seqname presenti in un determinato gene si assume come valore corretto per il confronto

*scr_id(gtf_row)*
- Ritorna il valore calcolato statisticamente del campo **source**
- Assumo che il campo **source** sia unico per tutto il file
- Il numero maggiore di source presenti nel file si assume come valore corretto per il confronto

*check_row_size(row)*
- Ritorna True se la riga esaminata ha il numero dei campi corretto (9 in questo caso), False altrimenti

*check_start_end(start, end, i)*
- Ritorna True se il campo **start** ed **end** rispettano le specifiche, False altrimenti
- start deve essere minore uguale ad end
- Devono essere valore interi maggiori di 0

*check_list(fte, vals)*
- Ritorna True se fte è presente nella lista vals, False altrimenti
- Viene utilizzata per il controllo dei campi **feature**, **strand** e **frame**

*check_score(score)*
- Ritorna True se il campo **score** rispetta le specifiche, False altrimenti

*check_attributes(row)*
- Controlla se sono presenti gli attributi obbligatori gene_id e transcript_id restituendo True, altrimenti False 

In [7]:
def seq_id(gtf_rows):
    seqid = dict()
    seqid_list = list()
    for row in gtf_rows:
        riga = row.rstrip().split('\t')
        if check_row_size(riga):
            gene_id = re.findall('gene_id\s+"([^"]+)";', riga[8])
            seqname = riga[0]
            if gene_id:
                seqid_list.append((gene_id[0], seqname))
    seqidC = collections.Counter(seqid_list)
    for values in seqidC.most_common():
        if values[0][0] not in list(seqid.keys()):
            seqid.update([values[0]])
    return seqid
             


def src_id(gtf_rows):
    srcid = list()
    for row in gtf_rows:
        riga = row.split('\t')
        if check_row_size(riga):
            srcid.append(riga[1])
    srcidC = collections.Counter(srcid)
    return srcidC.most_common(1)[0][0]  


def check_row_size(row):
    return len(row) == 9


def check_start_end(start, end, i):
    try:
        s = int(start)
        e = int(end)
        return (s <= e and s > 0 and e > 0)
    except ValueError:
        gtf_file_report.write("Errore alla linea: " + str(i) + ". campo <start> o <end> non corretto, non e' un intero\n");
        use_file = True
        return False
    


def check_list(fte, vals):
    return fte in vals


def check_score(score):
    if score == ".":
        return True
    else:
        try:
            s = int(score)
            return True
        except ValueError:
            try:
                s = float(score)
                return True
            except ValueError:
                return False
        return False



def check_attributes(row):
    transcript_id = re.findall('transcript_id\s+"([^"]+)";', row.rstrip().split('\t')[8])
    gene_id = re.findall('gene_id\s+"([^"]+)";', row.rstrip().split('\t')[8])
    return transcript_id and gene_id


In [8]:
# File per il report 

report_file_name = "report.txt"
gtf_file_report = open(report_file_name, 'wt')
gtf_file_report.write("REPORT FILE: " + gtf_file_name +"\n\n")

In [9]:
# indice di riga
i = 1

# variabile controllo errori o no
use_file = False

# liste dei valori ammessi
feature_list = ["CDS","start_codon","stop_codon","5UTR","3UTR","inter","inter_CNS","intron_CNS","exon"]
strand_list = ["+", "-"]
list_fme = ["0", "1", "2", "."]
list_ftr = ["CDS", "start_codon", "stop_codon"]

# dizionario (gene_id,seqname) per controllo statistico 
seqname_id = seq_id(gtf_file_rows)

# variabile contente source statisticamente stimato
src_id = src_id(gtf_file_rows)

# Controllo dell'intero file
for row in gtf_file_rows:
    riga = row.split('\t')
    if len(riga) > 1:
        if not check_row_size(riga):
            gtf_file_report.write("Errore alla linea: " + str(i) + ". Numero dei campi non rispettato\n");
            use_file = True
        else:
            if riga[1] != src_id:
                gtf_file_report.write("Errore alla linea: " + str(i) + ". Campo <source> non e' unico, trovato: " + riga[1] + "\n\tPossibile soluzione: " + src_id + " (stimato statisticamente)\n");
                use_file = True
            if not check_list(riga[2], feature_list):
                gtf_file_report.write("Errore alla linea: " + str(i) + ". Campo <feature> non corretto, trovato: " + riga[2] + "\n\tPossibile soluzione: " + str(feature_list) + "\n");
                use_file = True
            if not check_start_end(riga[3], riga[4], i):
                gtf_file_report.write("Errore alla linea: " + str(i) + ". <start> o <end> non corretto, trovato: " + riga[3] + " " + riga[4] + "\n\tPossibile soluzione: start <= end\n");
                use_file = True      
            if not check_score(riga[5]):
                gtf_file_report.write("Errore alla linea: " + str(i) + ". Campo <score> non corretto, trovato: " + riga[5] + "\n\tDovrebbe essere floating point number, integer o un punto\n");
                use_file = True
            if not check_list(riga[6], strand_list):
                gtf_file_report.write("Errore alla linea: " + str(i) + ". Campo <strand> non corretto, trovato: " + riga[6] + "\n\tPossibile soluzione: " + str(strand_list) + "\n");
                use_file = True
            if not check_list(riga[7], list_fme):
                gtf_file_report.write("Errore alla linea: " + str(i) + ". Campo <frame> non corretto, trovato: " + riga[7] + "\n\tPossibile soluzione: " + str(list_fme) + "\n");
                use_file = True
            elif (riga[2] not in list_ftr) and riga[7] != ".":
                gtf_file_report.write("Errore alla linea: " + str(i) + ". Campo <frame> non corretto, trovato: " + riga[7] + "\n\tPossibile soluzione: . \n");
                use_file = True
            elif check_list(riga[2], list_ftr) and not check_list(riga[7], list_fme[:-1]):
                gtf_file_report.write("Errore alla linea: " + str(i) + ". Campo <frame> non corretto, trovato: " + riga[7] + "\n\tPossibile soluzione: " + str(list_fme[:-1]) + "\n");
                use_file = True
            if not check_attributes(row):
                gtf_file_report.write("Errore alla linea: " + str(i) + ". Campo <attributes> non corretto, trovato: " + riga[8].rstrip() + "\n\tNon rispetta le specifiche\n");
                use_file = True
            elif riga[0] != seqname_id[re.findall('gene_id\s+"([^"]+)";', riga[8])[0]]:
                gtf_file_report.write("Errore alla linea: " + str(i) + ". Campo <seqname> non corretto, trovato: " + riga[0] + "\n\tPossibile soluzione: " + seqname_id[re.findall('gene_id\s+"([^"]+)";', riga[8])[0]] + " (stimato statisticamente)\n");
                use_file = True
    i += 1

# Nessun errore trovato
if (not use_file):
    gtf_file_report.write("File GTF corretto sintatticamente")
    

In [10]:
# Chiusura file analizzato e report

gtf_file_report.close()
gtf_input_file.close()